*This document contains the code need to run the simulations for chapter 3 of the PhD*

The script for all the scenarios are available and can be swithed 'on' and 'off' by deteleing an inserting hashtags (#)
for the relevant line of script.

The first step is to load the various software needed to run the simulations. These software are a combination of 
standard Python3 and OceanParcels libraries. These software packages also include built in OceanParcels kernels, such as AdvectionRK4 and AdvectionEE schemes. 

In [ ]:
# Load in software packages
%matplotlib inline
from parcels import FieldSet, Field, VectorField, ParticleFile, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, AdvectionEE, plotTrajectoriesFile, Variable, BrownianMotion2D
from parcels import ErrorCode
from parcels import rng as random
import numpy as np
import math
from datetime import timedelta, datetime
from operator import attrgetter

The next step is to design a custom class of particle that will contain the extra variables needed for the simulation such as distance travelled and age. The custom particle class can only be applied to a JITparticle. In the 'class' function of OceanParcels the various variables are defined. 

In addition to creating a custom class of particle, the kernels regarding the particles 'behaviour' must be defined (sinking, measuring distance and age, drag ect). This is performed through the 'def' function and the assocaited script. This allows for custom behaviour to be programmed. Note that the math script incorported into the kernels can only have basic numerical operators, which is a current limitation of the software. Also note, that in order to avoid an "error out of bounds" message, a kernel is created that deletes the particle if it leaves the study domain. This is standard partice when using OceanParcels and must be incorporated to successfully execute the simulation. The kernel for the different approaches to simulating drag are defined later.

In [ ]:
# Load in ocean model  Copernicus
filenames = {'U': "Copernicus/global-analysis-forecast-phy-001-024_1603882856077.nc",
             'V': "Copernicus/global-analysis-forecast-phy-001-024_1603882856077.nc"}
variables = {'U': 'uo',
             'V': 'vo'}
dimensions = {'lat': 'latitude',
              'lon': 'longitude',
              'time': 'time'}

fieldset = FieldSet.from_netcdf(filenames, variables, dimensions, allow_time_extrapolation=True)

In [ ]:
# Load in wind data

windfiles = {'U': "CERSAT/CERSAT-GLO-BLENDED_WIND_L4-V6-OBS_FULL_TIME_SERIE_1603883929498.nc",
             'V': "CERSAT/CERSAT-GLO-BLENDED_WIND_L4-V6-OBS_FULL_TIME_SERIE_1603883929498.nc"}
windvariables = {'U': 'eastward_wind',
                 'V': 'northward_wind'}
winddimensions = {'lat': 'lat',
                  'lon': 'lon',
                  'time': 'time'}

fieldsetwind = FieldSet.from_netcdf(windfiles, windvariables, winddimensions, allow_time_extrapolation=True)

In [ ]:
class SampleParticle(JITParticle):  # Define a new particle class that contains three extra variables
    age = Variable('age', dtype=np.float32, initial=0.) # initialise age
    distance = Variable('distance', initial=0., dtype=np.float32)  # the distance travelled
    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
                        initial=attrgetter('lon'))  # the previous longitude
    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
                        initial=attrgetter('lat'))  # the previous latitude.
    #uwind = Variable('uwind', initial=fieldset.U)
    #vwind = Variable('vwind', initial=fieldset.V)
    kelpvelU = Variable('kelpvelU',initial=fieldset.U)
    kelpvelV = Variable('kelpvelV',initial=fieldset.V)

def SampleDistance(particle, fieldset, time): # Function measuring distance
    # Calculate the distance in latitudinal direction (using 1.11e2 kilometer per degree latitude)
    lat_dist = (particle.lat - particle.prev_lat) * 1.11e2
    # Calculate the distance in longitudinal direction, using cosine(latitude) - spherical earth
    lon_dist = (particle.lon - particle.prev_lon) * 1.11e2 * math.cos(particle.lat * math.pi / 180)
    # Calculate the total Euclidean distance travelled by the particle
    particle.distance += math.sqrt(math.pow(lon_dist, 2) + math.pow(lat_dist, 2))

    particle.prev_lon = particle.lon  # Set the stored values for next iteration.
    particle.prev_lat = particle.lat
    
def SampleAge(particle, fieldset, time):
    particle.age = particle.dt

#def SampleWind(particle, fieldsetwind, time):
#    particle.uwind = fieldset.U[time, particle.depth, particle.lat, particle.lon]
#    particle.vwind = fieldset.V[time, particle.depth, particle.lat, particle.lon]

def SampleCurr(particle, fieldset, time):
    particle.currU = fieldset.U[time, particle.depth, particle.lat, particle.lon]
    particle.currV = fieldset.V[time, particle.depth, particle.lat, particle.lon]

def SampleKelpVel(particle, fieldset, time):
    particle.kelpvelU = fieldset.U[time, particle.depth, particle.lat, particle.lon]
    particle.kelpvelV = fieldset.V[time, particle.depth, particle.lat, particle.lon]
    
def DeleteParticle(particle, fieldset, time): # Function that deletes particle if it goes out of bounds to avoid OutOfBounds error
    particle.delete()

The next step is to define the dimensions of the data as well as tell OceanParcels where to attain the various data. Note, in models which have depth as a dimension, this must be defined in the script below in the "dimensions" section of the script. A field set constant is also added to define a sinking speed for the sinking behaviour of the particle and is acheievd through using the 'fieldset.add_constant' function. Next, the particle set is defined. The particle set contains information for the simulationsuch as, release coordinates, size, depth, repeat release ect. For information regarding the various methods for defining a particle set the reader is reffered to the OceanParcels documentation or manual.

In [ ]:
#This is to add brownian motion to the particle/s

fset_currents = FieldSet.from_netcdf(filenames, variables, dimensions)
fset_currents.add_periodic_halo(zonal=True)
size2D = (fset_currents.U.grid.ydim, fset_currents.U.grid.xdim)

fieldset.add_field(Field('Kh_zonal', data=10 * np.ones(size2D),
                         lon=fset_currents.U.grid.lon, lat=fset_currents.U.grid.lat,
                         mesh='spherical', allow_time_extrapolation=True))
fieldset.add_field(Field('Kh_meridional', data=10 * np.ones(size2D),
                         lon=fset_currents.U.grid.lon, lat=fset_currents.U.grid.lat,
                         mesh='spherical', allow_time_extrapolation=True))

In [ ]:
pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=SampleParticle,  # the type of particles (JITParticle/ScipyParticle/Custom)
                             lon=[18.3176], 
                             lat=[-34.1501],
                             depth=None,
                             repeatdt=3600,
                             time=(datetime(2018, 1, 1)))   

Note!! lonlatdepth_dtype: Floating precision for lon, lat, depth particle coordinates. It is either np.float32 or np.float64. Default is np.float32 if fieldset.U.interp_method is ‘linear’ and np.float64 if the interpolation method is ‘cgrid_velocity’

Before the execution of the simulation, the kernels for the various approaches to simulating drag are defined. There are two custom kernel behaviours, water drag and wind drag. The plant mass, cross-sectional area (Ac) and exposure to water and wind drag (expo & expoW) must be defined before each simulation. These parameterisations are dependent on the site of release.

Minumum: Ac = 122.97 & mass = 17.10
Mean : Ac = 228.61 & mass = 34.45
Maximum: Ac = 386.28 & mass = 48.65

In [ ]:
# Hydrodynamic drag kernel
def WaterDrag(particle, fieldset, time):
        (u,v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        Ac = 386.28  # cross-sectional area of plant, must be entered manually and calculated before hand
        mass = 48.65 # total mass of the kelp plant being simulated
        expo = 0.85
        waterdragU = ((0.5*1027.3*(u*u)*0.47*Ac)*expo)/mass # syntax for calculating the loss in velocity as a result of drag
        waterdragV = ((0.5*1027.3*(v*v)*0.47*Ac)*expo)/mass
        kelpvelU = (u - waterdragU)
        kelpvelV = (v - waterdragV)
        particle.lon  += kelpvelU * particle.dt   # overall velocity minus the loss in the velocity as a result of drag
        particle.lat  += kelpvelV * particle.dt   # overall velocity minus the loss in the velocity as a result of drag
        particle.kelpvelU = kelpvelU
        particle.kelpvelV = kelpvelV

In [ ]:
# Wind drag kernel
def WindDrag(particle, fieldsetwind, time):
        uw = fieldset.U[time, particle.depth, particle.lat, particle.lon]
        vw = fieldset.V[time, particle.depth, particle.lat, particle.lon]
        AcW = 386.28 # cross-sectional area of plant, must be entered manually and calculated before hand
        massW = 48.65 # total mass of the kelp plant being simulated
        expoW = 0.15
        winddragU = ((0.5*1.225*(uw*uw)*1.5*AcW)*expoW)/massW # syntax for calculating the loss in velocity as a result of drag
        winddragV = ((0.5*1.225*(vw*vw)*1.5*AcW)*expoW)/massW
        particle.lon += (uw - winddragU) * particle.dt  # overall velocity minus the loss in the velocity as a result of drag
        particle.lat += (vw - winddragV) * particle.dt  # overall velocity minus the loss in the velocity as a result of drag

Before the code can be executed, the output file must be defined and the kernels need to be 'cast' in order to activate them for the simulation. 

There are three different 'types' of cross-sectional areas and within in each type are different levels of wave and wind drag.

Finally, the code is executed with the timesteps defined and the user defined kernels included. 

In [ ]:
## cast kernels, select appropriate 'kernel set'

kernels = pset.Kernel(AdvectionRK4) + SampleDistance + SampleAge + BrownianMotion2D # for passive RK4 simulation
#kernels = pset.Kernel(AdvectionRK4) + SampleDistance + SampleKelpVel + BrownianMotion2D + WaterDrag # KelpFloat
#kernels = pset.Kernel(AdvectionRK4) + SampleDistance + SampleAge + SampleKelpVel + BrownianMotion2D + WaterDrag + WindDrag # KelpFloat


In [ ]:
## execute simulation

## define output file, select outfile according to simulation type

#pset.ParticleFile(name="RK4_PFloat.nc", outputdt=timedelta(hours=1)), 

#pset.ParticleFile(name="Min_KFloat_H100_W00.nc", outputdt=timedelta(hours=1)), 
#pset.ParticleFile(name="Min_KFloat_H095_W05.nc", outputdt=timedelta(hours=1)),  
#pset.ParticleFile(name="Min_KFloat_H090_W10.nc", outputdt=timedelta(hours=1)),  
#pset.ParticleFile(name="Min_KFloat_H085_W15.nc", outputdt=timedelta(hours=1)),  

#pset.ParticleFile(name="Mean_KFloat_H100_W00.nc", outputdt=timedelta(hours=1)), 
#pset.ParticleFile(name="Mean_KFloat_H095_W05.nc", outputdt=timedelta(hours=1)), 
#pset.ParticleFile(name="Mean_KFloat_H090_W10.nc", outputdt=timedelta(hours=1)), 
#pset.ParticleFile(name="Mean_KFloat_H085_W15.nc", outputdt=timedelta(hours=1)), 

#pset.ParticleFile(name="Max_KFloat_H100_W00.nc", outputdt=timedelta(hours=1)),  
#pset.ParticleFile(name="Max_KFloat_H095_W05.nc", outputdt=timedelta(hours=1)),  
#pset.ParticleFile(name="Max_KFloat_H090_W10.nc", outputdt=timedelta(hours=1)),  
#pset.ParticleFile(name="Max_KFloat_H085_W15.nc", outputdt=timedelta(hours=1)),  


pset.execute(kernels, 
             runtime=timedelta(days=30),
             dt=timedelta(hours=1),
             output_file=pset.ParticleFile(name="chap3_passivesim_acc.nc", outputdt=timedelta(hours=1)),  
             recovery = {ErrorCode.ErrorOutOfBounds: DeleteParticle})

Done! Now we can use the base plotting methods in Ocean Parcels to run some simple plotting to view the results of the simulation. The actual analysis and graphical outputs will be performed using R.

In [ ]:
# simple plot for viewing, use for checking simulation output

#plotTrajectoriesFile('chap3_sim_acc.nc')
plotTrajectoriesFile('chap3_passivesim_acc.nc')
#plotTrajectoriesFile('Max_KFloat_H100_W00.nc')


In [ ]:
plotTrajectoriesFile('chap3_passivesim_acc.nc', mode='movie2d_notebook')  

In [ ]:
pset.show(field='vector', show_time=datetime(2018, 1, 30, 0), domain={'N':-30, 'S':-35, 'E':21, 'W':15}, savefile = 'vector_day30', with_particles = False, vmax=0.8)

In [ ]:
pset.show(animation = True, field='vector', domain={'N':-30, 'S':-35, 'E':21, 'W':15}, savefile = 'vector_day', with_particles = False)